## Imports

In [1]:
import pandas as pd
import numpy as np
import datetime, random

from multiprocessing import Pool
import time
import os

## Reading in both dataframes; modifying Yu's dataframe to match mine and subsetting my dataframe on Yu's mrn/enc/time patients

In [2]:
yu = pd.read_csv(r'H:\Data\Standardized AKI definition\dataset\new aki flagger 1.csv')

In [3]:
%%time
df = pd.read_csv(r'H:\Data\Standardized AKI definition\dataset\output2.csv')
df['time'] = pd.to_datetime(df.time)
df = df.drop(['Unnamed: 0'], axis=1)

yu = pd.read_csv(r'H:\Data\Standardized AKI definition\dataset\new aki flagger 1.csv')
yu['mrn'] = yu.pat_mrn_id.str.strip('MR').astype('int') #indexing with integers is quicker
yu['enc'] = yu.enc_id
yu['time'] = pd.to_datetime(yu.time)

# yu.columns = ['mrn', #Renaming columns for ease
#              'pat_enc_id',
#              'time',
#              'creat',
#              'enc',
#              'running_aki_stage',
#              'historical_aki_stage',
#              'covid'] 

yu['running_aki'] = yu.running_aki_stage > 0 
yu['historical_aki'] = yu.historical_aki_stage > 0

yu = yu.groupby('enc').apply(lambda d: d[~d.time.duplicated()])
yu = yu.reset_index(drop=True)

print(yu.shape)

(169243, 23)
Wall time: 1min 21s


In [4]:
%%time
df_subset = df.set_index(['mrn', 'enc', 'time']).loc[yu.set_index(['mrn','enc','time']).index]
df_subset = df_subset.reset_index().groupby('enc').apply(lambda d: d[~d.time.duplicated()])
df_subset = df_subset.reset_index(drop=True)
print(np.all(df_subset.time == yu.time))
print(np.all(df_subset.enc == yu.enc))
print(np.all(df_subset.mrn == yu.mrn))
print('Subset shape:',df_subset.shape, 'Yu\'s df shape:',yu.shape)
df_subset.loc[df_subset.rollingwindow_aki.isnull(), 'rollingwindow_aki'] = 0
df_subset.loc[df_subset.backcalc_aki.isnull(), 'backcalc_aki'] = 0

True
True
True
Subset shape: (169243, 17) Yu's df shape: (169243, 23)
Wall time: 55.9 s


## Check mismatch 
-------

    (Reloading dataframes on Aug 4, 2020 - number of rows changed from 127254 to 169243 hmm.... still 100% match though)
###### Back-calc match: 169243 / 169243 ; 100% Match 

###### Rolling-window match: 169242 / 169234; 100% Match

In [5]:
print('Back-calc match:', (df_subset.backcalc_aki == yu.historical_aki).sum(), '/', df_subset.shape[0],
      ';',(df_subset.backcalc_aki == yu.historical_aki).sum() / df_subset.shape[0])

print('Rolling-window match:', (df_subset.rollingwindow_aki == yu.running_aki).sum(), '/', yu.shape[0],
      ';', (df_subset.rollingwindow_aki == yu.running_aki).sum() / yu.shape[0])

Back-calc match: 162661 / 169243 ; 0.961109174382397
Rolling-window match: 169242 / 169243 ; 0.9999940913361262


### Back-calc mismatch - FULLY MATCHED NOW

~~1248799, 4488514~~

In [6]:
yu_mismatch = yu.iloc[np.where(df_subset.backcalc_aki != yu.historical_aki)[0]] #we don't match on 75 patients
print(yu_mismatch.shape)
print(yu_mismatch.historical_aki.sum()) # He says they're all false

(6582, 23)
0


In [7]:
df_mismatch = df_subset.iloc[np.where(df_subset.backcalc_aki != yu.historical_aki)[0]]

### Rolling-window mismatch - FULLY MATCHED NOW

~~We mismatch on mrn: 2258332, 6492074, ...~~

In [8]:
yu_mismatch = yu.iloc[np.where(df_subset.rollingwindow_aki != yu.running_aki)[0]]
print(yu_mismatch.shape)
print(yu_mismatch.running_aki.sum()) #Again, he says all false

(1, 23)
0


In [9]:
#np.where(df_subset.loc[df_subset.mrn == 503537].rollingwindow_aki != yu.loc[yu.mrn == 503537].running_aki)